In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

In [ ]:
#Cargamos los datos de entrenamiento
train_df = pd.read_csv("data/original/Dataset_Salesforce_Predictive_Modelling_TRAIN.txt")

Veamos un ejemplo de los datos con los que contamos.

In [ ]:
train_df.head()

In [ ]:
train_df.columns

In [ ]:
train_df.dtypes

### Análisis de la variable dependiente.
Miremos con más detenimiento la variable a predecir.

In [ ]:
train_df["Poder_Adquisitivo"].describe()

In [ ]:
sns.distplot(train_df["Poder_Adquisitivo"])

In [ ]:
print("Skewness: %f" % train_df["Poder_Adquisitivo"].skew())
print("Kurtosis: %f" % train_df["Poder_Adquisitivo"].kurt())

In [ ]:
q1 = train_df["Poder_Adquisitivo"].quantile(0.25)
q3 = train_df["Poder_Adquisitivo"].quantile(0.75)
iqr = q3 - q1
fence_low  = q1 - 1.5 * iqr
fence_high = q3 + 1.5 * iqr

train_df_no_outliers = train_df.loc[(train_df["Poder_Adquisitivo"] > fence_low) & (train_df["Poder_Adquisitivo"] < fence_high)]
sns.distplot(train_df_no_outliers["Poder_Adquisitivo"])

In [ ]:
print("Skewness: %f" % train_df_no_outliers["Poder_Adquisitivo"].skew())
print("Kurtosis: %f" % train_df_no_outliers["Poder_Adquisitivo"].kurt())

Si bien puede que sea necesario estudiar más si esta es la manera más óptima de tratar los valores extremos, ahora presenta unos valores de skewness y kurtosis mucho más aceptables.

### Análisis de las variables independientes.

De las variables "independientes", sabemos que contamos con algunas que son de tipo categórico en vez de númerico. Empezemos explorando estas variables y preparandolas para su posterior uso.



In [ ]:
train_df["Socio_Demo_01"].value_counts()

In [ ]:
train_df["Socio_Demo_02"].value_counts()

Resulta inmediato convertir la segunda variable en un formato one-hot listo para ser usado por nuestros modelos. Más díficil es el caso de Socio_Dema_01 que cuenta que muchos valores que solo aparecen un número muy bajo de veces. Una primera aproximación puede ser usar por ejemplo los primeros 10 valores solo.

In [ ]:
topk_socio_01 = train_df["Socio_Demo_01"].value_counts()[:10]
topk_socio_01

Veamos si estas dos variables guardan alguna relación que se pueda ver a primera vista respecto al Poder adquisitivo.

In [ ]:
sns.violinplot(x=train_df_no_outliers["Socio_Demo_02"],y=train_df_no_outliers["Poder_Adquisitivo"])

In [ ]:
socio_01_keys = list(topk_socio_01.keys())
condition_array = [False] * len(train_df_no_outliers["Socio_Demo_01"])
for i in range(len(condition_array)):
    condition_array[i] = str(train_df_no_outliers["Socio_Demo_01"].iloc[i]) in socio_01_keys

sns.violinplot(x=train_df_no_outliers["Socio_Demo_01"].loc[condition_array],y=train_df_no_outliers["Poder_Adquisitivo"].loc[condition_array])

### Preparación de datos
Con todo lo visto anteriormente estamos listos para preparar los datos para la experimentación.



In [ ]:
def process_df_1(df,train = True):
    df = df.drop(labels=["ID_Customer"],axis=1)
    
    if train:
        #Remove putliers
        q1 = df["Poder_Adquisitivo"].quantile(0.25)
        q3 = df["Poder_Adquisitivo"].quantile(0.75)
        iqr = q3 - q1
        fence_low  = q1 - 1.5 * iqr
        fence_high = q3 + 1.5 * iqr

        df = df.loc[(df["Poder_Adquisitivo"] > fence_low) & (df["Poder_Adquisitivo"] < fence_high)]
    
        
    
    #Convert to one-hot

    #Socio_Demo_02
    c1=df["Socio_Demo_02"] == 1
    c2=df["Socio_Demo_02"] == 2
    df.insert(loc=len(df.columns), column="Socio_Demo_02_01", value=c1.astype(int))
    df.insert(loc=len(df.columns), column="Socio_Demo_02_02", value=c2.astype(int))
    
    df = df.drop(axis=1, columns=["Socio_Demo_02"])

    # Socio_Demo_01
    for key in socio_01_keys:
        on = df["Socio_Demo_01"] == key
        df.insert(loc=len(df.columns), column="Socio_Demo_01_"+key, value=on.astype(int))
    
    # Add option other
    condition_array = [False] * len(df["Socio_Demo_01"])
    for i in range(len(condition_array)):
        condition_array[i] = str(df["Socio_Demo_01"].iloc[i]) not in socio_01_keys
    df.insert(loc=len(df.columns), column="Socio_Demo_01_Other", value=condition_array)
    df["Socio_Demo_01_Other"] = df["Socio_Demo_01_Other"].astype(int)
    
    df = df.drop(axis=1, columns=["Socio_Demo_01"])
    
    # Ind_prod for now is kept as it is right now
    
    
    
    return df

In [ ]:
SEED = 4
K = 10

#Primero shuffleamos los datos
shuffled_data = train_df.sample(frac=1,replace=False,random_state=SEED)

kf = KFold(n_splits=K)
kf.get_n_splits(shuffled_data)

# Split and process data
splits=[]
for train_index, test_index in kf.split(shuffled_data):
    train_data = shuffled_data.loc[train_index]
    test_data = shuffled_data.loc[test_index]
    
    train_data_proc = process_df_1(train_data_c,train=True)
    test_data_proc = process_df_1(test_data,train=False)

    
    splits.append((train_data_proc,test_data_proc))


In [ ]:
def train_and_evaluate(model,splits):
    rmse = []
    mae = []
    for s in range(len(splits)):
        train_data_proc,test_data_proc = splits[s]
        
        x_train = train_data_proc.drop(labels=["Poder_Adquisitivo"],axis=1).as_matrix()
        y_train = train_data_proc["Poder_Adquisitivo"].as_matrix()


        x_test = test_data_proc.drop(labels=["Poder_Adquisitivo"], axis=1).as_matrix()
        y_test = test_data_proc["Poder_Adquisitivo"].as_matrix()
        
        model.fit(X=x_train,y=y_train)
        yhat = model.predict(X=x_test)
        
        rmse.append(math.sqrt(mean_squared_error(y_true=y_test, y_pred=yhat)))
        mae.append(mean_absolute_error(y_true=y_test,y_pred=yhat))
    return (rmse,mae)
        

In [ ]:
model = LinearRegression()
scores_rmse,scores_mae = train_and_evaluate(model,splits)


print("RMSE: %f" % np.mean(scores_rmse))
print("MAE: %f" % np.mean(scores_mae))


In [ ]:
########################
#### YOUR MODEL HERE ###
########################

# model = YourModel()
scores_rmse,scores_mae = train_and_evaluate(model,splits)


print("RMSE: %f" % np.mean(scores_rmse))
print("MAE: %f" % np.mean(scores_mae))
